In [3]:
# 실행하면 .env 파일의 OPENAI_API_KEY 를 자동으로 참조함.
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI()

b = chat.predict("How many planets are there?")

b

'There are currently eight known planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1) #temperature 0~1까지 1에 가까울 수록 무작위성 증가

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [5]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)


chat.predict_messages(prompt)

AIMessage(content='Γεια σου! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 17.000 χιλιόμετρα.')

#3.3 OutputParser and LCEL

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [2]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [3]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

In [4]:
chain = template | chat | CommaOutputParser() # chain은 template, chat, CommaOutputParser를 하나로 묶어서 순서대로 실행하는 파이프라인

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

#3.4 Chaining Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, # temperature 0~1까지 1에 가까울 수록 무작위성 증가
    streaming=True, # streaming=True로 설정하면 응답을 스트리밍 방식으로 받을 수 있음
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)


veg_chain = veg_chef_prompt | chat # veg_chain은 veg_chef_prompt와 chat을 하나로 묶어서 순서대로 실행하는 파이프라인

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})
# 입력: {"cuisine": "indian"}
# 1단계: chef_chain이 인도 요리 레시피를 생성
# 2단계: 생성된 레시피가 veg_chain으로 전달되어 채식 버전으로 변환
# 출력: 채식 인도 요리 레시피

Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Here's a recipe for a classic Indian dish called Butter Chicken:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust according to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt, to taste
- Fresh cilantro, for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute to release the flavors.
4. Add the chicken pieces to the pan and cook until they are lightly brow

AIMessageChunk(content="Great choice! Butter Chicken is a delicious and popular Indian dish. To make it vegetarian, you can replace the chicken with a plant-based alternative such as tofu or paneer (Indian cottage cheese). Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 2 tablespoons butter (or vegan butter for a vegan version)\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust according to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup heavy cream (or coconut cream for a vegan version)\n- Salt, to taste\n- Fresh cilantro, for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix toget

#4.1 FewShotPromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1, # 낮은 temperature로 일관성 있는 답변 생성
    streaming=True, # 스트리밍으로 실시간 답변 출력
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}") # 예시 프롬프트 템플릿 생성

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt, # 각 예시의 형식
    examples=examples, # 학습용 예시 데이터
    suffix="Human: What do you know about {country}?", # 실제 질문 템플릿
    input_variables=["country"], # 입력 변수 정의
)
# Few-Shot Learning의 장점 : AI가 특정 도메인에서 일관된 형식으로 답변해야 할 때 매우 유용
# 일관된 형식: 예시를 통해 원하는 답변 형식을 명확히 지정
# 적응성: 새로운 입력에 대해서도 학습된 패턴 적용
# 제어 가능성: 예시를 통해 답변 스타일과 내용 구조 제어

chain = prompt | chat

chain.invoke({"country": "Turkey"})

AI:
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira')

#4.2 FewShotChatMessagePromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France", # 질문에 들어갈 변수
        "answer": """ 
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"), # 사용자 메시지
        ("ai", "{answer}"), # AI의 답변 메시지
    ]
)

# 앞서 정의한 example_prompt 형식과 examples 데이터를 조합하여 Few-shot 템플릿을 생성
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt, # Few-shot 예시들이 여기에 삽입됨
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')

#4.3 LengthBasedExampleSelector

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    # 예시 데이터를 초기화
    def __init__(self, examples):
        self.examples = examples

    # 새로운 예시를 동적으로 추가
    def add_example(self, example):
        self.examples.append(example)

    # 핵심 메서드 - 입력에 따라 사용할 예시를 선택
    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector, # 선택적 예시 사용
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

#4.4 Serialization and Composition

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


####################

from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")
# prompt = load_prompt("./prompt.yaml")

prompt.format(country="Korea")

####################

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


# 1. pipeline_prompts의 각 템플릿이 먼저 실행됨
# 2. 그 결과가 final_prompt의 플레이스홀더에 삽입됨
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrrrg matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothin' beats a good ol' pirate feast on the high seas! Arrrrg!

AIMessageChunk(content="Arrrrg matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothin' beats a good ol' pirate feast on the high seas! Arrrrg!")

#4.5 Caching

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db")) # OpenAI API 호출 결과를 SQLite 데이터베이스에 저장하여 중복 요청을 방지


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [4]:
chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

#4.6 Serialization

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


# with 문 내부의 모든 OpenAI API 호출을 추적
# 각 요청의 토큰 사용량과 비용을 누적 계산
with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)

Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of sugar

Instructions:
1. Rinse the rice thoroughly and soak it in water for at least 1 hour.
2. Drain the rice and place it in a steamer basket. Steam the rice for about 30 minutes or until it is cooked through.
3. Transfer the cooked rice to a large bowl and let it cool to room temperature.
4. In a separate bowl, mix the nuruk and sugar with water to create a paste.
5. Add the rice to the nuruk paste and mix well to combine.
6. Cover the bowl with a clean cloth and let it ferment in a warm place for about 3-4 days.
7. After the fermentation process is complete, strain the mixture through a cheesecloth to remove any solids.
8. Transfer the liquid to a clean bottle and store it in the refrigerator until ready to serve.

Enjoy your homemade soju! 

Ingredients:
- 4 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cups warm water
- 2 tablespoons sugar
- 2 teaspoons salt


In [ ]:
from langchain.llms.openai import OpenAI

chat = OpenAI(
model_name="gpt-3.5-turbo",
temperature=0.1,
max_tokens=450,  # 응답의 최대 토큰 수
)

# chat으로 설정한 모델을 JSON 파일로 저장
chat.save("model.json")

/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
import json
from langchain.llms.openai import OpenAI

# model.json 파일을 읽기 모드로 열기
# json.load()로 JSON 데이터를 Python 딕셔너리로 변환
with open("model.json", "r") as f:
	config = json.load(f)

# config 변수에 모델 설정 저장
chat = OpenAI(**config)

chat

/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_kwargs={'temperature': 0.1, 'max_tokens': 450, '_type': 'openai-chat'})

#5.0 ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory

# ConversationBufferMemory은 이전 대화 내용을 모두 저장
memory = ConversationBufferMemory(return_messages=True) # return_messages=True로 설정하면 chat model 작업에 사용할 수 있는 형태로 저장함.


memory.save_context(
{"input":"hi"}
, {"output": "hou are you?"}
)

memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'), AIMessage(content='hou are you?')]}

In [11]:
memory.save_context(
{"input":"hi"}
, {"output": "hou are you?"}
)

memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content='hou are you?'),
  HumanMessage(content='hi'),
  AIMessage(content='hou are you?')]}

In [ ]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough


model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)


def load_memory(_):
    x = memory.load_memory_variables({})
    return {"history": x["history"]}


chain = RunnablePassthrough.assign(history=load_memory) | prompt | model

inputs = {"message": "hi im bob"}
response = chain.invoke(inputs)
response

#5.1 ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

# 마지막 K개의 상호 작용만 사용. 이는 버퍼가 너무 커지지 않도록 가장 최근 상호 작용의 슬라이딩 윈도우를 유지함.
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=3,  # 최근 3개의 대화만 저장
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

#5.2 ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm) # 대화를 요약하여 저장하는 메모리, 요약을 생성하기 위해 LLM을 사용함 -> 대화가 길어질수록 요약이 유용함.


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("South Korea is so pretty", "I wish I could go!!!")
add_message("I love the k-food", "Me too, Korean food is amazing!")
get_history()

{'history': 'Nicolas introduces himself as living in South Korea. The AI responds with enthusiasm at the information, expressing a desire to visit the country because it is so pretty. The human mentions their love for Korean food, to which the AI agrees, saying Korean food is amazing.'}

#5.3 ConversationSummaryBufferMemory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# 토큰이 30개 이하면 → 그대로 저장
# 토큰이 30개 초과하면 → AI가 요약해서 저장
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=30, # 최대 토큰 수를 30으로 설정하여 요약의 길이를 제한, 작게 설정 → 자주 요약됨
    return_messages=True,
)

# 대화를 메모리에 추가
def add_message(input, output): 
    memory.save_context({"input": input}, {"output": output})

# 저장된 내용 확인
def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("South Korea is so pretty", "I wish I could go!!!")
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

get_history()



{'history': [SystemMessage(content="Nicolas from South Korea introduces himself to the AI, who responds enthusiastically. The human mentions how pretty South Korea is, and the AI expresses a desire to visit the country. When asked about the distance between Korea and Argentina, the AI admits it doesn't know, but thinks it's super far."),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

#5.4 ConversationKGMemory

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# KG = Knowledge Graph (지식 그래프)
# 일반 메모리는 대화를 그냥 저장하지만, KG Memory는 사실들을 연결해서 기억한다.
# 대화에서 중요한 사실들을 자동으로 찾아내서 저장한다.
# 사실 추출:
# - Nicolas는 사람이다
# - Nicolas는 한국에 산다
# - 한국은 나라다

# 연결 관계:
# Nicolas --[살고있음]--> 한국
# Nicolas --[이름은]--> "Nicolas"
memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"input": "who is Nicolas"})


{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea.')]}

In [ ]:
add_message("Nicolas likes kimchi", "Wow that is so cool!") # 이 대화로 인해 "Nicolas"와 "kimchi" 사이의 관계가 추가됨
memory.load_memory_variables({"inputs": "what does nicolas like"})


{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea. Nicolas likes kimchi.')]}

#5.5 Memory on LLMChain

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=10,
    memory_key="chat_history", # 프롬프트 템플릿에서 {chat_history}로 접근
)

# {chat_history}: 메모리에서 가져온 이전 대화
# {question}: 현재 사용자 질문
# You:: AI 응답 시작 프롬프트
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is Nico
    You:


> Finished chain.


'Hello Nico! How can I assist you today?'

In [2]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    System: The human introduces themselves as Nico. The AI greets Nico and asks how it can assist them today.
    Human:I live in Seoul
    You:


> Finished chain.


"That's great to hear, Nico! How can I assist you today regarding living in Seoul?"

In [3]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    System: Nico introduces themselves as living in Seoul. The AI greets Nico and asks how it can assist them today regarding living in Seoul.
    Human:What is my name?
    You:


> Finished chain.


'Your name is Nico. How can I assist you today regarding living in Seoul, Nico?'

#5.6 Chat Based Memory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=15,
    memory_key="chat_history",
    return_messages=True, # 🔥 핵심: 메시지 객체로 반환 -> HumanMessage, AIMessage 객체로 반환 -> AI가 대화 패턴을 더 잘 이해
)

# 구조화된 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"), # MessagesPlaceholder는 메모리의 메시지들을 프롬프트에 자동으로 삽입
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico

> Finished chain.


'Hello Nico! How can I assist you today?'

In [5]:
chain.predict(question="I live in Seoul")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
System: The human introduces themselves as Nico. The AI greets Nico and asks how it can assist them today.
Human: I live in Seoul

> Finished chain.


"That's great to know, Nico! Seoul is a vibrant and bustling city with a rich history and culture. Is there anything specific you'd like to know or discuss about Seoul today?"

In [6]:
chain.predict(question="What is my name?")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
System: Nico introduces themselves and mentions they live in Seoul. The AI greets Nico and acknowledges the vibrant nature of Seoul, offering to discuss or provide information about the city.
Human: What is my name?

> Finished chain.


'Your name is Nico. How can I assist you today, Nico?'

#5.7 LCEL Based Memory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"), # MessagesPlaceholder는 메모리의 메시지들을 프롬프트에 자동으로 삽입
        ("human", "{question}"),
    ]
)

# 메모리에서 대화 기록을 가져옴
# ["history"]로 메시지 리스트만 추출
def load_memory(_):
    return memory.load_memory_variables({})["history"]

# RunnablePassthrough은 메모리에서 가져온 대화 기록을 프롬프트에 전달
# assign() 메서드를 사용하여 load_memory 함수를 실행하고 결과를 "history"
# 변수에 할당함
# 이 "history" 변수는 프롬프트 템플릿에서 MessagesPlaceholder로 사용됨
# 따라서 메모리에서 가져온 대화 기록이 프롬프트에 자동으로 삽입됨   
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    ) # 질문과 답변을 메모리에 수동으로 저장
    print(result)

In [2]:
invoke_chain("My name is nico")

content='Hello Nico! How can I assist you today?'


In [3]:
invoke_chain("What is my name?")

content='Your name is Nico.'


#6.2 Tiktoken

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

# 텍스트를 청크로 분할
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",# 줄바꿈 기준으로 분할
    chunk_size=1000,# 청크당 최대 1000 토큰
    chunk_overlap=100,# 인접 청크 간 100 토큰 중복
)
# UnstructuredFileLoader는 다양한 파일 형식을 로드할 수 있는 로더
loader = UnstructuredFileLoader("./files/chapter_one.docx")

# load_and_split() 메서드는 파일을 로드하고 지정된 텍스트 분할기를 사용하여 텍스트를 청크로 분할
loader.load_and_split(text_splitter=splitter) 

[nltk_data] Downloading package punkt to /Users/yongjae/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[Document(page_content="Part 1, Chapter 1\nPart One\n1\nIt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a varic